In [1]:
import syft
from syft.core import torch
from syft.core.worker import LocalWorker, BaseWorker

hook = syft.core.torch_.hook.TorchHook()

local = hook.worker
remote = LocalWorker(id=1, hook=hook)

hook.add_worker(remote.id,remote)

Hooking into Torch...
Overloading complete.


In [10]:
x = torch.ones(2,2)

In [11]:
y = x.send_(remote)

In [12]:
remote.objects

{399399793: [torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x10fa60588>]],
 2539219195: [torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x10fa60588>]],
 4282721710: [torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x10fa60588>]]}

In [13]:
z = y + y

In [14]:
z

[torch.FloatTensor - Locations:[<syft.core.worker.LocalWorker object at 0x10fa60588>]]

In [15]:
z.get_()


 2  2
 2  2
[torch.FloatTensor of size 2x2]

In [16]:
y.get_()


 1  1
 1  1
[torch.FloatTensor of size 2x2]